In [40]:
import pandas as pd
import sympy as sp
import math as mt
import numpy as np

In [60]:
def truncar(numero, decimales):
    """   
    :numero: El número a truncar
    :param decimales: Cuántos decimales mantener
    :return: número truncado en número n de decimales, ejemplo: original= 0.335, 0.33 a 2 decimales truncados
    """
    factor = 10 ** decimales
    return mt.trunc(numero * factor) / factor
def coeficientes_minimos_cuadrados_general(func_str, x_lista, y_lista, valores_iniciales):
    """
    :param func_str: string, función base para el modelado
    :param x_lista: lista de valores de x
    :param y_lista: lista de valores de y
    :param valores_iniciales: lista con valores iniciales, ej: [1, 0.1]
    :return: diccionario {a: valor_est, b: valor_est, ...}
    """
    f = sp.sympify(func_str)

    simbolos = list(f.free_symbols) #Obtiene las variables independientes y coeficientes de la función

    x = sp.Symbol('x')
    if x not in simbolos:
        raise ValueError("La función debe contener el símbolo 'x' como variable independiente.")
    
    parametros = [s for s in simbolos if s != x] #Filtra y guarda únicamente los coeficientes

    if len(parametros) != len(valores_iniciales):
        raise ValueError("El número de valores iniciales debe coincidir con el número de parámetros.")

    S = 0
    #Lógica para las derivadas parciales y resolucion del sistema de ecuaciones
    for xi, yi in zip(x_lista, y_lista):
        S += (yi - f.subs(x, xi))**2

    ecuaciones = [sp.diff(S, p) for p in parametros]

    solucion = sp.nsolve(
        ecuaciones,
        parametros,
        valores_iniciales,
        tol=1e-8
    )

    return {str(parametros[i]): float(solucion[i]) for i in range(len(parametros))} #Crea diccionario con las soluciones

def minimos_cuadrados_general_simbolico(func_str, coeficientes, decimales):
    """
    :param func_str: string, función original
    :param coeficientes: diccionario, valor de los coeficientes de la función
    :param decimales: entero, número de decimales a truncar
    :return: string con la función y sus respectivos coeficientes truncados según decimales
    """
    for clave, valor in coeficientes.items():
        func_str = func_str.replace(str(clave), str(truncar(valor, decimales)))   
    return func_str

def minimos_cuadrados_general_numerico(func_str, coeficientes):
    """
    :param func_str: string, función original
    :param coeficientes: lista con los coeficientes del modelo
    :return: función lambda, capaz de realizar operaciones del tipo f(arg), donde arg es un valor y f devuelve otro
    """
    x = sp.Symbol("x")
    func_num = sp.sympify(func_str)
    func_num = func_num.subs(coeficientes)
    return sp.lambdify(x, func_num, "numpy")

def crear_modelo(entrada, modelo_base):
    try:
        df = pd.read_csv("datos_ajuste.csv")
    except FileNotFoundError:  # Archivo no encontrado
        print("Archivo no encontrado:", entrada)  # Mensaje de error
        return None
    except Exception as e:  # Captura de otro tipo de errores
        print("Error al leer el CSV:", e)  # Mensaje de error
        return None
    
    datos_x = [i + 1 for i in range(len(df))]
    datos_y = df['sales_week'].tolist()
    valores_iniciales = [0] * len([s for s in list(sp.sympify(modelo_base).free_symbols) if s !=sp.Symbol('x')])
    coeficientes = coeficientes_minimos_cuadrados_general(modelo_base, datos_x, datos_y, valores_iniciales)
    return minimos_cuadrados_general_numerico(modelo_base, coeficientes), minimos_cuadrados_general_simbolico(modelo_base, coeficientes, 5)
_, modelo_1 =crear_modelo("datos_ajuste.csv", "a*x+b")
_, modelo_2 = crear_modelo("datos_ajuste.csv", "a*x**2+b*x+c")
print(modelo_1, modelo_2)

4957.04751*x+313743.88309 -2.41086*x**2+5424.75467*x+298543.40035


In [61]:
def metricas_modelo(entrada, modelo_base, valores_iniciales=None):
    try:
        df = pd.read_csv(entrada)
    except FileNotFoundError:
        print("Archivo no encontrado:", entrada)
        return None
    except Exception as e:
        print("Error al leer el CSV:", e)
        return None

    # y: datos reales
    y = pd.to_numeric(df["sales_week"], errors="coerce").dropna().to_numpy(dtype=float)
    n = len(y)
    x = np.arange(n, dtype=float)
    num_parametros = len([s for s in list(sp.sympify(modelo_base).free_symbols) if s != sp.Symbol('x')])
    if valores_iniciales is None:
        valores_iniciales = [0] * num_parametros
    if len(valores_iniciales) != num_parametros:
        print("El número de valores iniciales no coincide con el número de parámetros.")
        return None
    coeficientes = coeficientes_minimos_cuadrados_general(modelo_base, x.tolist(), y.tolist(), valores_iniciales)
    f = minimos_cuadrados_general_numerico(modelo_base, coeficientes)
    
    # y_sombrero: predicción del modelo en los mismos x de los datos
    y_sombrero = np.array(f(x), dtype=float)
    ## Error del ajuste
    residuo = y - y_sombrero                 # Diferencia entre valor real y estimado
    sse = float(np.sum(residuo**2))          # Suma de errores cuadráticos
    mse = float(sse / n)                     # Error cuadrático medio
    rmse = float(np.sqrt(mse))               # Raíz del error cuadrático medio
    mae = float(np.mean(np.abs(residuo)))    # Error absoluto medio
    ## Calidad del ajuste
    sst = float(np.sum((y - np.mean(y))**2)) # Variabilidad total de los datos
    r = float(np.corrcoef(y, y_sombrero)[0, 1]) #Coeficiente de correlación
    r2 = float(1.0 - sse/sst) if sst > 0 else float("nan")  # Coeficiente de determinación
    ## Complejidad del modelo
    cte = 1e-12                              # Evita log(0)
    aic = float(n * np.log((sse / n) + cte) + 2 * num_parametros)       # Criterio AIC
    bic = float(n * np.log((sse / n) + cte) + num_parametros * np.log(n)) # Criterio BIC
    return {"SSE": sse, "RMSE": rmse, "MAE": mae, "R2": r2, "r": r, "AIC": aic, "BIC": bic}
print(metricas_modelo("datos_ajuste.csv", "b*x+c"))
print(metricas_modelo("datos_ajuste.csv", "a*x**2+b*x+c"))


{'SSE': 7383874851495.624, 'RMSE': 195597.59436063227, 'MAE': 154312.4876358682, 'R2': 0.6659554865473172, 'r': 0.8160609576173329, 'AIC': 4706.952488766656, 'BIC': 4713.477869144465}
{'SSE': 7375229148333.217, 'RMSE': 195483.0492145647, 'MAE': 154258.35080725452, 'R2': 0.6663466158343687, 'r': 0.8163005670917844, 'AIC': 4708.726374647367, 'BIC': 4718.514445214082}
